In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim
import numpy as np
from torch.utils.data import random_split
from xgboost.sklearn import XGBClassifier


In [2]:
def load_data(filename):
    data = np.loadtxt(filename, delimiter=',', dtype=np.float32)
    x = data[:, 0:-2]
    y = data[:, [-1]] # after 3 months (-2) and 6 months (-1)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [206]:
# target to fix code


class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_size), nn.Sigmoid())
        self.layer2 = nn.Sequential(nn.Linear(hidden_size, output_size), nn.Sigmoid())
        #self.layer3 = nn.Sequential(nn.Linear(hidden_size, output_size), nn.Sigmoid())
        self.dropout = nn.Dropout(0.005)
        #self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_size), nn.Tanh())
        self.activation = torch.nn.ELU()
        #self.dropout = nn.Dropout(0.1)
        #self.layer2 = nn.Sequential(nn.Linear(hidden_size, output_size), nn.Sigmoid())
        #self.dropout = nn.Dropout(0.1)
        #self.activation = torch.nn.ReLU()
        
        
        

        
    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.activation(x)
        x = self.layer2(x)
        #x = self.dropout(x)
        #x = self.layer3(x)
        
        
        
        return x # return model

In [207]:
x_train, y_train = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_training_dataset.csv')
x_test, y_test = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_test_dataset.csv')

In [208]:
y_train

tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
      

In [209]:
x_test

tensor([[0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.2000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.3000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 0.7000, 0.6111, 1.0000, 0.1652, 0.4800],
        [0.0042, 0.4179, 0.2535, 0.5000, 0.2500, 0.2727, 0.3986, 0.0000, 0.0000,
         0.2388, 0.6000, 0.5000, 1.0000, 0.1739, 0.7500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 1.0000, 0.7222, 0.0000, 0.1739, 0.0000],
        [0.3208, 0.4467, 0.2676, 0.5000, 0.7500, 0.2727, 0.7489, 0.2812, 0.3559,
         0.1939, 1.0000, 0.7222, 0.0000, 0.3043, 0.7500],
        [0.0084, 0.5389, 0.0000, 1.0000, 0.2500, 0.1818, 0.6514, 0.0000, 0.0735,
         0.3152, 0.4000, 1.0000, 1.0000, 0.2609, 0.0000],
        [0.0168, 0.7176, 0.

In [210]:
x_train.shape

torch.Size([103, 15])

In [211]:


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test) # test set에는 transform만 사용하기


x_train=torch.Tensor(x_train)
x_test=torch.Tensor(x_test)

In [212]:
model = FNN(input_size=15, hidden_size=300, output_size=1)
criterion = nn.BCELoss()
#optimizer = optim.Adam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix
optimizer = optim.Adam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix

In [213]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0986,  0.2259, -0.0984,  ..., -0.2437,  0.1808,  0.0531],
        [ 0.0668, -0.0041, -0.0720,  ...,  0.1869,  0.1217,  0.0428],
        [ 0.0936,  0.2104,  0.0488,  ...,  0.0726,  0.0217,  0.1102],
        ...,
        [-0.2134, -0.0318, -0.1159,  ..., -0.2429,  0.1059,  0.2066],
        [ 0.0203, -0.1323,  0.1972,  ...,  0.2256, -0.0997,  0.0894],
        [-0.1529, -0.2416, -0.0394,  ...,  0.2220, -0.2208, -0.0221]],
       requires_grad=True)
Parameter containing:
tensor([-0.2199,  0.2314, -0.0441,  0.1054, -0.1151, -0.1255,  0.0732, -0.0843,
         0.1915,  0.0121,  0.0700, -0.1899,  0.1048, -0.1034,  0.1050,  0.0204,
         0.0782,  0.0265, -0.1827,  0.1895,  0.1645,  0.2492,  0.0270,  0.2052,
         0.0107,  0.0578, -0.1996, -0.2011,  0.0731, -0.1621, -0.2370, -0.2126,
        -0.1993, -0.0892, -0.1348, -0.1998,  0.1911,  0.0811,  0.2398, -0.0670,
         0.2534,  0.1557,  0.2581,  0.1255, -0.2103, -0.1672, -0.2414, -0.1974,
        -0.1695

In [214]:
# assuming y is torch.Tensor

In [215]:
num_epochs = 1001

for epoch in range(num_epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')
        

with torch.no_grad(): 
    train_accuracy = (model(x_train).gt(0.5).float() == y_train).float().mean().item()
    test_accuracy = (model(x_test).gt(0.5).float() == y_test).float().mean().item()
    
    

print(f'\nTrain Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
#Train Accuracy: 0.9709 -> start
#Test Accuracy: 0.9000 -> start

Epoch [0/1001], Loss: 0.6508
Epoch [200/1001], Loss: 0.1309
Epoch [400/1001], Loss: 0.0481
Epoch [600/1001], Loss: 0.0320
Epoch [800/1001], Loss: 0.0220
Epoch [1000/1001], Loss: 0.0279

Train Accuracy: 1.0000
Test Accuracy: 0.9500


In [216]:
model(x_test).gt(0.5)

tensor([[False],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False]])

In [217]:
model(x_test).gt(0.5).float()

tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [218]:
y_test

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]])